# 1. Import libraries



In [ ]:
import pandas as pd
import numpy as np
import missingno
from collections import Counter

# Data visualisation
import seaborn as sns
import matplotlib.pyplot as plt

# Machine learning models
from sklearn.linear_model import LogisticRegression, Perceptron, SGDClassifier
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
from catboost import CatBoostClassifier

from sklearn.ensemble import VotingClassifier

# Model evaluation
from sklearn.model_selection import cross_val_score

# Hyperparameter tuning
from sklearn.model_selection import GridSearchCV

# Remove warnings
import warnings
warnings.filterwarnings('ignore')

# 2. Import and read data

Now import and read the 3 datasets as outlined in the introduction.

In [ ]:
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")
ss = pd.read_csv("gender_submission.csv")

train_copy = pd.read_csv("train.csv")
test_copy = pd.read_csv("test.csv")


Let's have a look at the datasets.

In [ ]:
train.head()

In [ ]:
test.head()

# 3. Data description

Here I will outline the definitions of the columns in the titanic dataset. You can find this information under the [data](https://www.kaggle.com/c/titanic/data) tab of the competition page.

- Survived: 0 = Did not survive, 1 = Survived

- Pclass: Ticket class where 1 = First class, 2 = Second class, 3 = Third class. This can also be seen as a proxy for socio-economic status.

- Sex: Male or female

- Age: Age in years, fractional if less than 1

- SibSp: Number of siblings or spouses aboard the titanic

- Parch: Number of parents or children aboard the titanic

- Ticket: Passenger ticket number

- Fare: Passenger fare

- Cabin: Cabin number

- Embarked: Point of embarkation where C = Cherbourg, Q = Queenstown, S = Southampton


The data has been split into two groups:
- training set (train.csv)
- test set (test.csv)

The training set includes passengers survival status (also know as the ground truth from the titanic tragedy) which along with other features like gender, class, fare and pclass is used to create the machine learning model.

The test set should be used to see how well the model performs on unseen data. The test set does not provide passengers survival status. We are going to use our model to predict passenger survival status.

This is clearly a <font color='red'>__Classification problem__.</font> In predictive analytics, when the <font color='red'>__target__</font> is a categorical variable, we are in a category of tasks known as <font color='red'>__classification tasks.__</font>

# 4. Exploratory Data Analysis (EDA)

Exploratory data analysis is the process of visualising and analysing data to extract insights. In other words, we want to summarise important characteristics and trends in our data in order to gain a better understanding of our dataset.

__Exploratory Data Analysis__ refers to the critical process of performing initial investigations on data so as to discover patterns, to spot anomalies, to test hypothesis and to check assumptions with the help of summary statistics and graphical representations.

In summary, it's an approach to analyzing data sets to summarize their main characteristics, often with visual methods.

## 4.1 Data types,data shapes, missing data and summary statistics

### 4.1.1.Data Types

In [ ]:
# Non-null count and data types of the training

train.info()

The training-set has 891 rows and 11 features + the __target variable (survived).__ 2 of the features are floats, 5 are integers and 5 are objects.

### 4.1.2.Data Shape

In [ ]:
print("Training set shape: ", train.shape)
print("Test set shape: ", test.shape)

Note that the test set has one column less than training set, the Survived column. This is because Survived is our response variable, or sometimes called a target variable. Our job is to analyse the data in the training set and predict the survival of the passengers in the test set.

What about sample submission?

### 4.1.3.Missing Values

In [ ]:
# Missing data in training set by columns

train.isnull().sum().sort_values(ascending = False)

In [ ]:
# Missing data in test set by columns 

test.isnull().sum().sort_values(ascending = False)

In [ ]:
#percentages of missing values in training data
total = train.isnull().sum().sort_values(ascending=False)
percent_1 = train.isnull().sum()/train.isnull().count()*100
percent_2 = (round(percent_1, 1)).sort_values(ascending=False)
missing_data = pd.concat([total, percent_2], axis=1, keys=['Total', '%'])
missing_data.head(13)

Seems like Age, Cabin and Embarked colummns in the training set have missing data  while Age, Fare and Cabin in the test set have missing data. 

In the training set:
The __'Embarked'__ feature has only 2 missing values, which can easily be filled or dropped. It will be much more tricky to deal with the __‘Age’__ feature, which has 177 missing values. The __‘Cabin’__ feature needs further investigation, but it looks like we might want to drop it from the dataset since 77% is missing.

### 4.1.4.Statistics

In [ ]:
# Summary statistics for training set 

train.describe()

__.describe()__ gives an understanding of the central tendencies of the numeric data.

- Above we can see that __38% out of the training-set survived the Titanic.__ 
- We can also see that the passenger age range from __0.4 to 80 years old.__
- We can already detect some features that contain __missing values__, like the ‘Age’ feature (714 out of 891 total).
- There's an __outlier__ for the 'Fare' price because of the differences between the 75th percentile, standard deviation, and the max value (512). We might want to drop that value.

In [ ]:
# Summary statistics for test set 

test.describe()

## 4.2 Feature analysis

A dataframe is made up of rows and columns. Number of rows correspond to the number of observations in our dataset whereas columns, sometimes called features, represent characteristics that help describe these observations. In our dataset, rows are the passengers on the titanic whereas columns are the features that describe the passengers like their age, gender etc.

Before we move on, it is also important to note the difference between a categorical variable and a numerical variable. Categorical variables, as the name suggests, have values belonging to one of two or more categories and there is usually no intrinsic ordering to these categories. An example of this in our data is the Sex feature. Every passenger is distinctly classified as either male or female. Numerical variables, on the other hand, have a continuous distribution. Some examples of numerical variables are the Age and Fare features.

Knowing if a feature is a numerical variable or categorical variable helps us structure our analysis more properly. For instance, it doesn't make sense to calculate the average of a categorical variable such as gender simply because gender is a binary classification and therefore has no intrinsic ordering to its values.

In this next section of the notebook, we will analyse the features in our dataset individually and see how they correlate with survival probability.

### 4.2.1 Categorical variables

Categorical variables in our dataset are Sex, Pclass and Embarked.

#### 4.2.1.1.Categorical variable: Sex

In [ ]:
# Value counts of the sex column

train['Sex'].value_counts(dropna = False)

Observation: There are more male passengers than female passengers on titanic

In [ ]:
# Mean of survival by sex

train[['Sex', 'Survived']].groupby('Sex', as_index = False).mean().sort_values(by = 'Survived', ascending = False)

In [ ]:
sns.barplot(x = 'Sex', y ='Survived', data = train)
plt.ylabel('Survival Probability')
plt.title('Survival Probability by Gender')

Observation: Female passengers are more likely to survive

#### 4.2.1.2.Categorical variable: Pclass

In [ ]:
# Value counts of the Pclass column 

train['Pclass'].value_counts(dropna = False)

In [ ]:
# Mean of survival by passenger class

train[['Pclass', 'Survived']].groupby(['Pclass'], as_index = False).mean().sort_values(by = 'Survived', ascending = False)

In [ ]:
plt.subplots(figsize=(10,8))
ax=sns.kdeplot(train.loc[(train['Survived'] == 0),'Pclass'],shade=True,color='r',label='Not Survived')
ax.legend()
ax=sns.kdeplot(train.loc[(train['Survived'] == 1),'Pclass'],shade=True,color='b',label='Survived')
ax.legend()

plt.title("Passenger Class Distribution - Survived vs Non-Survived", fontsize = 25)
labels = ['First', 'Second', 'Third']
plt.xticks(sorted(train.Pclass.unique()),labels);

In [ ]:
sns.barplot(x = 'Pclass', y ='Survived', data = train)
plt.ylabel('Survival Probability')
plt.title('Survival Probability by Passenger Class')

# Comment: Survival probability decrease with passenger class, first class passengers are prioritised during evacuation

notebook 5
Observation: The graphs above clearly shows that economic status (Pclass) played an important role regarding the potential survival of the Titanic passengers. First class passengers had a much higher chance of survival than passengers in the 3rd class. We note that:

63% of the 1st class passengers survived the Titanic wreck
48% of the 2nd class passengers survived
Only 24% of the 3rd class passengers survived

#### 4.2.1.3.Categorical variables combined: Sex and Plass

In [ ]:
# Survival by gender and passenger class

g = sns.factorplot(x = 'Pclass', y = 'Survived', hue = 'Sex', data = train, kind = 'bar')
g.despine(left = True)
plt.ylabel('Survival Probability')
plt.title('Survival Probability by Sex and Passenger Class')

#Aiden did this
Observation: This indicates that in every class females where always more likely to survive. The higher the class the greater the chnace of survival of the men. 

#### 4.2.1.4.Categorical variable: Embarked

In [ ]:
# Value counts of the Embarked column 
#NAN is the missing values in Embarked

train['Embarked'].value_counts(dropna = False)

In [ ]:
# Mean of survival by point of embarkation

train[['Embarked', 'Survived']].groupby(['Embarked'], as_index = False).mean().sort_values(by = 'Survived', ascending = False)

In [ ]:
sns.barplot(x = 'Embarked', y ='Survived', data = train)
plt.ylabel('Survival Probability')
plt.title('Survival Probability by Point of Embarkation')

Survival probability is highest for location C and lowest for location S.

Is there a reason for this occurence? We can formulate a hypothesis whereby the majority of the first class passengers have embarked from location C and because they have a highest survival probability, this has resulted in location C having a highest survival probability. Alternatively, there could have been more third class passengers that embarked from location S and because they have the lowest survival probability, this has caused location S to have the lowest survival probability.

Let us now test this hypothesis.

#### 4.2.1.5.Categorical variable combined: Embarked and Class

In [ ]:
sns.factorplot('Pclass', col = 'Embarked', data = train, kind = 'count')

Our hypothesis appears to be true. Location S has the most third class passengers whereas location C has the most first class passengers. 

### 4.2.2 Numerical variables

Numerical variables in our dataset are SibSp, Parch, Age and Fare.

#### 4.2.2.1.Detect outliers in numerical variables

Outliers are data points that have extreme values and they do not conform with the majority of the data. It is important to address this because outliers tend to skew our data towards extremes and can cause inaccurate model predictions. I will use the Tukey method to detect these outliers which will later be dropped under data processing

In [ ]:
def detect_outliers(df, n, features):
    """"
    This function will loop through a list of features and detect outliers in each one of those features. In each
    loop, a data point is deemed an outlier if it is less than the first quartile minus the outlier step or exceeds
    third quartile plus the outlier step. The outlier step is defined as 1.5 times the interquartile range. Once the 
    outliers have been determined for one feature, their indices will be stored in a list before proceeding to the next
    feature and the process repeats until the very last feature is completed. Finally, using the list with outlier 
    indices, we will count the frequencies of the index numbers and return them if their frequency exceeds n times.    
    """

    #can relace outliers with the mean/median (even grouped by title) and then add a column indicating if it was an outlier as the fact it was an outlier might be related to survival
    outlier_indices = [] 
    for col in features: 
        Q1 = np.percentile(df[col], 25)
        Q3 = np.percentile(df[col], 75)
        IQR = Q3 - Q1
        outlier_step = 1.5 * IQR 
        outlier_list_col = df[(df[col] < Q1 - outlier_step) | (df[col] > Q3 + outlier_step)].index
        outlier_indices.extend(outlier_list_col) 
    outlier_indices = Counter(outlier_indices)
    multiple_outliers = list(key for key, value in outlier_indices.items() if value > n) 
    return multiple_outliers

outliers_to_drop = detect_outliers(train, 2, ['Age', 'SibSp', 'Parch', 'Fare'])#these are the four numerical variables discussed
outliers_to_drop_test = detect_outliers(test, 2, ['Age', 'SibSp', 'Parch', 'Fare'])#these are the four numerical variables discussed
# outliers_to_drop = detect_outliers(train, 1, ['Age','Fare'])#these are the four numerical variables discussed
# outliers_to_drop_test = detect_outliers(test, 1, ['Age', 'Fare'])#these are the four numerical variables discussed

print("We will drop these {} indices: ".format(len(outliers_to_drop)), outliers_to_drop) 

In [ ]:
# Outliers in numerical variables
#allows us to look at the 10 rows identified above as rows containing outliers
train.loc[outliers_to_drop, :]

#### 4.2.2.2.Numerical variables correlation with survival

In [ ]:
df_num = train[['Age','SibSp','Parch','Fare']]
plt.subplots(figsize = (12,6))
sns.heatmap(df_num.corr(), annot=True,cmap="RdBu")
plt.title("Correlations Among Numeric Features", fontsize = 18);

We notice from the heatmap above that:
- __Parents and sibling like to travel together <font color='blue'>(light blue squares)__</font> Therefore it will useful to create a isAlone and a family size feature
- __Age has a high negative correlation with number of siblings__

#### 4.2.2.3.Numerical variable: SibSp

In [ ]:
# Value counts of the SibSp column 
#numbner of siblings
train['SibSp'].value_counts(dropna = False)

In [ ]:
# Mean of survival by SibSp

train[['SibSp', 'Survived']].groupby('SibSp', as_index = False).mean().sort_values(by = 'Survived', ascending = False)

In [ ]:
sns.barplot(x = 'SibSp', y ='Survived', data = train)
plt.ylabel('Survival Probability')
plt.title('Survival Probability by SibSp')

#### 4.2.2.4.Numerical variable: Parch

In [ ]:
# Value counts of the Parch column 

train['Parch'].value_counts(dropna = False)

In [ ]:
# Mean of survival by Parch

train[['Parch', 'Survived']].groupby('Parch', as_index = False).mean().sort_values(by = 'Survived', ascending = False)

In [ ]:
sns.barplot(x = 'Parch', y ='Survived', data = train)
plt.ylabel('Survival Probability')
plt.title('Survival Probability by Parch')

#### 4.2.2.5.Numerical variable: Age

In [ ]:
# Null values in Age column 

train['Age'].isnull().sum()

In [ ]:
# Passenger age distribution

sns.distplot(train['Age'], label = 'Skewness: %.2f'%(train['Age'].skew()))
plt.legend(loc = 'best')
plt.title('Passenger Age Distribution')

In [ ]:
# Age distribution by survival

g = sns.FacetGrid(train, col = 'Survived')
g.map(sns.distplot, 'Age')

In [ ]:
sns.kdeplot(train['Age'][train['Survived'] == 0], label = 'Did not survive')
sns.kdeplot(train['Age'][train['Survived'] == 1], label = 'Survived')
plt.xlabel('Age')
plt.title('Passenger Age Distribution by Survival')

#### 4.2.2.6.Numerical variable: Fare

In [ ]:
# Passenger fare distribution

sns.distplot(train['Fare'], label = 'Skewness: %.2f'%(train['Fare'].skew()))
plt.legend(loc = 'best')
plt.ylabel('Passenger Fare Distribution')

Fare seems to have a high skewness. We will address this issue later on in the notebook via log transformation. 

### 4.2.3 Correlation between categorical and numerical

#### 4.2.3.1.All variables

In [ ]:
sns.heatmap(train[['Survived', 'SibSp', 'Parch', 'Age', 'Fare','Pclass']].corr(), annot = True, fmt = '.2f', cmap = 'coolwarm')

Observation: Fare seems to be the only feature that has a substantial correlation with survival

#### 4.2.3.2.Age and Sex

In [ ]:
survived = 'survived'
not_survived = 'not survived'

fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(15, 5))

women = train[train['Sex']=='female']
men = train[train['Sex']=='male']

# Plot Female Survived vs Not-Survived distribution
ax = sns.histplot(women[women['Survived']==1].Age.dropna(), bins=20, label = survived, ax = axes[0],color='b', kde=True)
ax = sns.histplot(women[women['Survived']==0].Age.dropna(), bins=20, label = not_survived, ax = axes[0],color='r', kde=True)
ax.legend()
ax.set_title('Female')

# Plot Male Survived vs Not-Survived distribution
ax = sns.histplot(men[men['Survived']==1].Age.dropna(), bins=20, label = survived, ax = axes[1],color='b', kde=True)
ax = sns.histplot(men[men['Survived']==0].Age.dropna(), bins=20, label = not_survived, ax = axes[1],color='r', kde=True)
ax.legend()
ax.set_title('Male');

We can see that __men__ have a higher probability of survival when they are between __18 and 35 years old.__ For __women,__ the survival chances are higher between __15 and 40 years old.__

For men the probability of survival is very low between the __ages of 5 and 18__, and __after 35__, but that isn’t true for women. Another thing to note is that __infants have a higher probability of survival.__

# 5. Data preprocessing

Getting the dataset in a form to be modelled and trained. This includes:
- Dealing with ouliers
- Drop and fill missing values
- Data transformation 
- Feature engineering
- Feature encoding

## 5.1 Remove Outliers

In [ ]:
# Drop outliers 

print("Train Set Before: {} rows".format(len(train)))
#train = train.drop(outliers_to_drop, axis = 0).reset_index(drop = True)
print("Train Set After: {} rows".format(len(train)))
print("Test Set Before: {} rows".format(len(test)))
# test = test.drop(outliers_to_drop_test, axis = 0).reset_index(drop = True)
print("Test Set After: {} rows".format(len(test)))

## 5.2 Drop and fill missing values

In [ ]:
# Drop ticket and cabin features from training and test set as they are unique or missing many values
train = train.drop(['Ticket', 'Cabin'], axis = 1)
test = test.drop(['Ticket', 'Cabin'], axis = 1)

I have decided to drop both ticket and cabin for simplicity of this tutorial but if you have the time, I would recommend going through them and see if they can help improve your model.

In [ ]:
train.isnull().sum().sort_values(ascending = False)

In [ ]:
# Fill missing value in Embarked with mode as only 3 values
mode = train['Embarked'].dropna().mode()[0]
train['Embarked'].fillna(mode, inplace = True)

In [ ]:
test.isnull().sum().sort_values(ascending = False)

In [ ]:
# Fill missing value for Fare 
median = test['Fare'].dropna().median()
test['Fare'].fillna(median, inplace = True)

In [ ]:
# Check where indeces of missing ages are
age_nan_indices_train = list(train[train['Age'].isnull()].index)
len(age_nan_indices_train)
age_nan_indices_test = list(test[test['Age'].isnull()].index)


Age is negatively correlated with SibSp, Parch and Pclass as shown in section 4. Loop through each those rows and fill the missing age with their median. Othwerise fill with the Age median.

In [ ]:
for index in age_nan_indices_train:
    median_age = train['Age'].median()
    predict_age = train['Age'][(train['SibSp'] == train.iloc[index]['SibSp']) 
                                 & (train['Parch'] == train.iloc[index]['Parch'])
                                 & (train['Pclass'] == train.iloc[index]["Pclass"])].median()
    if np.isnan(predict_age):
        train['Age'].iloc[index] = median_age
    else:
        train['Age'].iloc[index] = predict_age
combine = pd.concat([train, test], axis = 0).reset_index(drop = True)
median_age = combine['Age'].median()
for index in age_nan_indices_test:
    #use larger sample to fill test data 
    test['Age'].iloc[index] = median_age  

In [ ]:
# Make sure there are no more missing ages 
print(train['Age'].isnull().sum())
test['Age'].isnull().sum()

## 5.3 Data transformation

Recall that our passenger fare column has a very high positive skewness. Therefore, we will apply a log transformation to address this issue.

In [ ]:
#  fare distribution

sns.distplot(train['Fare'], label = 'Skewness: %.2f'%(train['Fare'].skew()))
plt.legend(loc = 'best')
plt.title('Passenger Fare Distribution')

In [ ]:
# In order to reduce skewness in fare, apply log transformation 
train['Fare'] = train['Fare'].map(lambda x: np.log(x) if x > 0 else 0)
test['Fare'] = test['Fare'].map(lambda x: np.log(x) if x > 0 else 0)

In [ ]:
# After log transformation

sns.distplot(train['Fare'], label = 'Skewness: %.2f'%(train['Fare'].skew()))
plt.legend(loc = 'best')
plt.title('Fare Distribution After Log Transformation')

## 5.4 Feature engineering

We create new features from existing features to obtain an improved model.

### 5.4.1 Title

In [ ]:
train.head()

In [ ]:
#Title from name column
train['Title'] = [name.split(',')[1].split('.')[0].strip() for name in train['Name']]
train[['Name', 'Title']].head()
test['Title'] = [name.split(',')[1].split('.')[0].strip() for name in test['Name']]
test[['Name', 'Title']].head()

In [ ]:
# Value counts of Title
train['Title'].value_counts()

In [ ]:
# visualise the testing titles
test['Title'].value_counts()

In [ ]:
# Simplify Title as there are several unique itles that do not necessarily have a trend

train['Title'] = train['Title'].replace(['Dr', 'Rev', 'Col', 'Major', 'Lady', 'Jonkheer', 'Don', 'Capt', 'the Countess',
                                             'Sir'], 'Rare')
train['Title'] = train['Title'].replace(['Mlle', 'Ms'], 'Miss')
train['Title'] = train['Title'].replace('Mme', 'Mrs')

test['Title'] = test['Title'].replace(['Dr', 'Rev', 'Col',  'Capt', 'Dona'], 'Rare')
test['Title'] = test['Title'].replace(['Ms'], 'Miss')


In [ ]:
# Drop name column as title has been extracted


train = train.drop('Name', axis = 1)
train.head()

test = test.drop('Name', axis = 1)
test.head()

### 5.4.2 IsAlone

In [ ]:
train['FamilySize'] = train['SibSp'] + train['Parch'] + 1
train[['SibSp', 'Parch', 'FamilySize']].head()

test['FamilySize'] = test['SibSp'] + test['Parch'] + 1
test[['SibSp', 'Parch', 'FamilySize']].head()

In [ ]:
# Create IsAlone feature as familySize may have more information than we need, leading to overfitting

train['IsAlone'] = 0
train.loc[train['FamilySize'] == 1, 'IsAlone'] = 1

test['IsAlone'] = 0
test.loc[test['FamilySize'] == 1, 'IsAlone'] = 1

In [ ]:
# Drop SibSp, Parch and FamilySize as this is contained in isAlone

train = train.drop(['SibSp', 'Parch','FamilySize'], axis = 1)
test = test.drop(['SibSp', 'Parch','FamilySize'], axis = 1)
train.head()

### 5.4.3 Age*Class

First convert Age into an ordinal variable. Group Ages into 5 age bands 

In [ ]:

train['AgeBand'] = pd.cut(train['Age'], 5)
test['AgeBand'] = pd.cut(test['Age'], 5)

In [ ]:
train.loc[train['Age'] <= 16.136, 'Age'] = 0
train.loc[(train['Age'] > 16.136) & (train['Age'] <= 32.102), 'Age'] = 1
train.loc[(train['Age'] > 32.102) & (train['Age'] <= 48.068), 'Age'] = 2
train.loc[(train['Age'] > 48.068) & (train['Age'] <= 64.034), 'Age'] = 3
train.loc[train['Age'] > 64.034 , 'Age'] = 4

test.loc[test['Age'] <= 16.136, 'Age'] = 0
test.loc[(test['Age'] > 16.136) & (test['Age'] <= 32.102), 'Age'] = 1
test.loc[(test['Age'] > 32.102) & (test['Age'] <= 48.068), 'Age'] = 2
test.loc[(test['Age'] > 48.068) & (test['Age'] <= 64.034), 'Age'] = 3
test.loc[test['Age'] > 64.034 , 'Age'] = 4

# Drop age band feature
train = train.drop('AgeBand', axis = 1)
test = test.drop('AgeBand', axis = 1)

In [ ]:
# Convert ordinal Age into integer
train['Age'] = train['Age'].astype('int')
test['Age'] = test['Age'].astype('int')
train['Age'].dtype

In [ ]:
# Create Age*Class

train['Age*Class'] = train['Age'] * train['Pclass']
test['Age*Class'] = test['Age'] * test['Pclass']
train[['Age', 'Pclass', 'Age*Class']].head()

In [ ]:
# Bin Fare 
train['FareBand'] = pd.qcut(train['Fare'], 4)
test['FareBand'] = pd.qcut(test['Fare'], 4)
train['FareBand'].head(10)


In [ ]:
#ordinal encoding, simliar to age
train.loc[train['Fare'] <= 2.066, 'Fare'] = 0
train.loc[(train['Fare'] > 2.066) & (train['Fare'] <= 2.671), 'Fare'] = 1
train.loc[(train['Fare'] > 2.671) & (train['Fare'] <= 3.418), 'Fare'] = 2
train.loc[train['Fare'] > 3.418, 'Fare'] = 3

test.loc[test['Fare'] <= 2.066, 'Fare'] = 0
test.loc[(test['Fare'] > 2.066) & (test['Fare'] <= 2.671), 'Fare'] = 1
test.loc[(test['Fare'] > 2.671) & (test['Fare'] <= 3.418), 'Fare'] = 2
test.loc[test['Fare'] > 3.418, 'Fare'] = 3

In [ ]:
train = train.drop([ 'FareBand'], axis = 1)
test = test.drop(['FareBand'], axis = 1)

In [ ]:
# Convert Fare into integer
train['Fare'] = train['Fare'].astype('int')
test['Fare'] = test['Fare'].astype('int')

## 5.5 Feature encoding 

Variables must be numeric to use for machine learning. Age and Fare were done when Binning. 

In [ ]:
train.head()

In [ ]:
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
label = LabelEncoder() 
train['Embarked'] = label.fit_transform(train['Embarked'])
test['Embarked'] = label.fit_transform(test['Embarked'])
train['Title'] = label.fit_transform(train['Title'])
test['Title'] = label.fit_transform(test['Title'])
train['Sex'] = train['Sex'].map({'male': 0, 'female': 1})
test['Sex'] = test['Sex'].map({'male': 0, 'female': 1})

train.head()

In [ ]:
train = train.drop('PassengerId', axis = 1)
train.head()

In [ ]:
train['Survived'] = train['Survived'].astype('int')
train.head()

In [ ]:
test.head()

# 6. Modelling

Scikit-learn is one of the most popular libraries for machine learning in Python and that is what we will use in the modelling part of this project. 

Since Titanic is a classfication problem, we will need to use classfication models, also known as classifiers, to train on our model to make predictions. I highly recommend checking out this scikit-learn [documentation](https://scikit-learn.org/stable/index.html) for more information on the different machine learning models available in their library. I have chosen the following classifiers for the job:

- Logistic regression
- Support vector machines
- K-nearest neighbours
- Gaussian naive bayes
- Perceptron
- Linear SVC
- Stochastic gradient descent
- Decision tree
- Random forest
- CatBoost

In this section of the notebook, I will fit the models to the training set as outlined above and evaluate their accuracy at making predictions. Once the best model is determined, I will also do hyperparameter tuning to further boost the performance of the best model.

## 6.1 Split training data

We need to first split our training data into independent variables or predictor variables, represented by X as well as  dependent variable or response variable, represented by Y.

Y_train is the survived column in our training set and X_train is the other columns in the training set excluding the Survived column. Our models will learn to classify survival, Y_train based on all X_train and make predictions on X_test.

In [ ]:
X_train = train.drop('Survived', axis = 1)
Y_train = train['Survived']
X_test = test.drop('PassengerId', axis = 1).copy()#why only drop now
print("X_train shape: ", X_train.shape)
print("Y_train shape: ", Y_train.shape)
print("X_test shape: ", X_test.shape)

## 6.2 Fit model to data and make predictions

This requires 3 simple steps: instantiate the model, fit the model to the training set and predict the data in test set. 

### 6.2.1 Logistic regression

 Explanation (not to be included in final submision): In section 6.2, we are training our models using the ENTIRE training set (every row that has a survive column). The models are UNTUNED.. We then calculate the accuracy of each model for the TRAINING set data. In other words we  determine how accurate each model is when it is asked to predict the outcome  (survival)  for the passengers on which it was trained. High scores might be an inidcation of which algorithms are likely to work well for predicting survival for passenges in the test set(this is the ultimate goal), although high scores could also indicate overfitting which is bad . These scores are summarised in the next section

In [ ]:
#rael
logreg = LogisticRegression()
LGtrained=logreg.fit(X_train, Y_train)
acc_log = round(logreg.score(X_train, Y_train) * 100, 2)


### 6.2.2 Support vector machines

In [ ]:
#rael
svc = SVC()
svc.fit(X_train, Y_train)
acc_svc = round(svc.score(X_train, Y_train) * 100, 2)


### 6.2.3 K-nearest neighbours (KNN)

In [ ]:
#rael

knn = KNeighborsClassifier(n_neighbors = 5)
KNNtrained=knn.fit(X_train, Y_train)
acc_knn = round(knn.score(X_train, Y_train) * 100, 2)

### 6.2.4 Gaussian naive bayes

In [ ]:
#rael
gaussian = GaussianNB()
gaussian.fit(X_train, Y_train)
acc_gaussian = round(gaussian.score(X_train, Y_train) * 100, 2)

### 6.2.5 Perceptron

In [ ]:
#rael
perceptron = Perceptron()
perceptron.fit(X_train, Y_train)
acc_perceptron = round(perceptron.score(X_train, Y_train) * 100, 2)

### 6.2.6 Linear SVC

In [ ]:
#rael
linear_svc = LinearSVC()
linear_svc.fit(X_train, Y_train)
acc_linear_svc = round(linear_svc.score(X_train, Y_train) * 100, 2)

### 6.2.7 Stochastic gradient descent

In [ ]:
#rael
sgd = SGDClassifier()
sgd.fit(X_train, Y_train)
acc_sgd = round(sgd.score(X_train, Y_train) * 100, 2)

### 6.2.8 Decision tree

In [ ]:
#rael
decision_tree = DecisionTreeClassifier()
decision_tree.fit(X_train, Y_train)
acc_decision_tree = round(decision_tree.score(X_train, Y_train) * 100, 2)

### 6.2.9 Random forest

In [ ]:
#rael
random_forest = RandomForestClassifier(n_estimators = 100)
random_forest.fit(X_train, Y_train)
acc_random_forest = round(random_forest.score(X_train, Y_train) * 100, 2)

### 6.2.10 CatBoost

In [ ]:
#rael
catboost = CatBoostClassifier()
catboost.fit(X_train, Y_train)
acc_catboost = round(catboost.score(X_train, Y_train) * 100, 2)

In [ ]:
#rael
#MLP
mlp = MLPClassifier()
mlp.fit(X_train, Y_train)
acc_mlp = round(catboost.score(X_train, Y_train) * 100, 2)

In [ ]:
#rael
#acc_catboost

## 6.3 Model evaluation and hyperparameter tuning

Once all our models have been trained, the next step is to assess the performance of these models and select the one which has the highest prediction accuracy. 

### 6.3.1 Training accuracy

Training accuracy shows how well our model has learned from the training set. 

Internal comment: Viewing and summarising the scores calcualted above for each algorithm. These models have have not yet been tuned

In [ ]:
models = pd.DataFrame({'Model': ['Support Vector Machines', 'KNN', 'Logistic Regression', 
                                 'Random Forest', 'Naive Bayes', 'Perceptron', 'Stochastic Gradient Decent', 
                                 'Linear SVC', 'Decision Tree', 'CatBoost','MLP'],
                       'Score': [acc_svc, acc_knn, acc_log, acc_random_forest, acc_gaussian, acc_perceptron,
                                 acc_sgd, acc_linear_svc, acc_decision_tree, acc_catboost, acc_mlp]})

models.sort_values(by = 'Score', ascending = False, ignore_index = True)

### 6.3.2 K-fold cross validation

It is important to not get too carried away with models with impressive training accuracy as what we should focus on instead is the model's ability to predict out-of-samples data, in other words, data our model has not seen before.

This is where k-fold cross validation comes in. K-fold cross validation is a technique whereby a subset of our training set is kept aside and will act as holdout set for testing purposes. Here is a great [video](https://www.youtube.com/watch?v=fSytzGwwBVw) explaining the concept in more detail. 

In [ ]:
# Create a list which contains classifiers 

classifiers = []
classifiers.append(LogisticRegression())
classifiers.append(SVC())
classifiers.append(KNeighborsClassifier(n_neighbors = 5))
classifiers.append(GaussianNB())
classifiers.append(Perceptron())
classifiers.append(LinearSVC())
classifiers.append(SGDClassifier())
classifiers.append(DecisionTreeClassifier())
classifiers.append(RandomForestClassifier())
classifiers.append(CatBoostClassifier())
classifiers.append(MLPClassifier())


len(classifiers)

In [89]:
# Create a list which contains cross validation results for each classifier

cv_results = []
for classifier in classifiers:#each result has 10 subcompoents for each section of the data that was made test if cv equals 10
    cv_results.append(cross_val_score(classifier, X_train, Y_train, scoring = 'accuracy', cv = 10))#try other cv's

957:	learn: 0.3217501	total: 932ms	remaining: 40.9ms
958:	learn: 0.3216330	total: 935ms	remaining: 40ms
959:	learn: 0.3216183	total: 935ms	remaining: 39ms
960:	learn: 0.3215836	total: 936ms	remaining: 38ms
961:	learn: 0.3215535	total: 936ms	remaining: 37ms
962:	learn: 0.3215101	total: 937ms	remaining: 36ms
963:	learn: 0.3214479	total: 938ms	remaining: 35ms
964:	learn: 0.3214449	total: 938ms	remaining: 34ms
965:	learn: 0.3214111	total: 938ms	remaining: 33ms
966:	learn: 0.3213749	total: 938ms	remaining: 32ms
967:	learn: 0.3213463	total: 939ms	remaining: 31.1ms
968:	learn: 0.3213205	total: 940ms	remaining: 30.1ms
969:	learn: 0.3212934	total: 941ms	remaining: 29.1ms
970:	learn: 0.3212585	total: 941ms	remaining: 28.1ms
971:	learn: 0.3212235	total: 945ms	remaining: 27.2ms
972:	learn: 0.3211785	total: 945ms	remaining: 26.2ms
973:	learn: 0.3211749	total: 946ms	remaining: 25.2ms
974:	learn: 0.3211440	total: 946ms	remaining: 24.3ms
975:	learn: 0.3211157	total: 947ms	remaining: 23.3ms
976:	learn:

In [ ]:
# Mean and standard deviation of cross validation results for each classifier  

cv_mean = []
cv_std = []
for cv_result in cv_results:
    cv_mean.append(cv_result.mean())
    cv_std.append(cv_result.std())

In [ ]:
cv_res = pd.DataFrame({'Cross Validation Mean': cv_mean, 'Cross Validation Std': cv_std, 'Algorithm': ['Logistic Regression', 'Support Vector Machines', 'KNN', 'Gausian Naive Bayes', 'Perceptron', 'Linear SVC', 'Stochastic Gradient Descent', 'Decision Tree', 'Random Forest', 'CatBoost','MLP']})
cv_res.sort_values(by = 'Cross Validation Mean', ascending = False, ignore_index = True)

In [ ]:
sns.barplot('Cross Validation Mean', 'Algorithm', data = cv_res, order = cv_res.sort_values(by = 'Cross Validation Mean', ascending = False)['Algorithm'], palette = 'Set3', **{'xerr': cv_std})
plt.ylabel('Algorithm')
plt.title('Cross Validation Scores')

As we can see, support vector machines has the highest cross validation mean and thus we will proceed with this model.

### 6.3.3 Hyperparameter tuning for SVM

Hyperparameter tuning is the process of tuning the parameters of a model. Here I will tune the parameters of support vector classifier using GridSearchCV.

In [ ]:
# param_grid = {'n_neighbors': [1,2,3,4,5,6],  
#              # 'gamma': [1, 0.1, 0.01, 0.001, 0.0001], 
#              # 'kernel': ['rbf']}  
# }
# grid = GridSearchCV(KNeighborsClassifier(), param_grid, refit = True, verbose = 3) 

# grid.fit(X_train, Y_train) 

In [ ]:
param_grid = {'alpha': [0,1e-5,1e-4,1e-3,1e-2,1e-1,1],  
             # 'gamma': [1, 0.1, 0.01, 0.001, 0.0001], 
             # 'kernel': ['rbf']}  
}
grid = GridSearchCV(MLPClassifier(), param_grid, refit = True, verbose = 3) 

grid.fit(X_train, Y_train) 

In [ ]:
print("Best parameters: ", grid.best_params_) 
print("Best estimator: ", grid.best_estimator_)# what is this

In [ ]:
# Training accuracy

# #svc = KNeighborsClassifier(C = 100, gamma = 0.01, kernel = 'rbf')
# knn = KNeighborsClassifier(n_neighbors=6)
# knn.fit(X_train,Y_train)
# #svc.fit(X_train, Y_train)
# Y_pred = knn.predict(X_test)
# acc_svc = round(knn.score(X_train, Y_train) * 100, 2)
# acc_svc

In [ ]:
#svc = KNeighborsClassifier(C = 100, gamma = 0.01, kernel = 'rbf')
mlp = grid.best_estimator_
trainedAndTunedlMLP=mlp.fit(X_train,Y_train)
#svc.fit(X_train, Y_train)
Y_pred = mlp.predict(X_test)
acc_svc = round(mlp.score(X_train, Y_train) * 100, 2)
acc_svc

In [ ]:
# Mean cross validation score

cross_val_score(mlp, X_train, Y_train, scoring = 'accuracy', cv = 5).mean()

Our mean cross validation score improved slightly.

In [ ]:
# Survival predictions by support vector classifier

Y_pred

In [ ]:
len(Y_pred)

### 6.3.4 Ensembles

In [ ]:
best_trained_MLP =trainedAndTunedlMLP #still not sure what this estimator is
trained_knn = KNNtrained
trained_lg = LGtrained


voting_clf_soft = VotingClassifier(estimators = [('mlp',best_trained_MLP),('knn',trained_knn),('lg',trained_lg)], voting = 'soft') 



print('voting_clf_soft :',cross_val_score(voting_clf_soft,X_train,Y_train,cv=5))
print('voting_clf_soft mean :',cross_val_score(voting_clf_soft,X_train,Y_train,cv=5).mean())

params = {'weights' : [[1,1,1],[1,2,1],[1,1,2],[2,1,1],[2,2,1],[1,2,2],[2,1,2]]}

vote_weight = GridSearchCV(voting_clf_soft, param_grid = params, cv = 5, verbose = True, n_jobs = -1)
best_clf_weight = vote_weight.fit(X_train, Y_train)
voting_clf_sub = best_clf_weight.best_estimator_.predict(X_test)

voting_clf_soft.fit(X_train, Y_train)
Y_pred =  voting_clf_soft.predict(X_test).astype(int)

# 7. Preparing data for submission

In [ ]:
ss.head()

In [ ]:
ss.shape

We want our submission dataframe to have 418 rows and 2 columns, PassengerId and Survived. 

In [ ]:
# Create submission dataframe

submit = pd.DataFrame({'PassengerId': test['PassengerId'], 'Survived': Y_pred})
submit.head()

In [ ]:
submit.shape

Our dataframe is ready for submission!

In [ ]:
# Create and save csv file 

submit.to_csv("submission.csv", index = False)

# 8. Possible extensions to improve model accuracy

1. Analyse ticket and cabin features
    - Do these features help predict passenger survival?
    - If yes, consider including them in the training set instead of dropping
2. Come up with alternative features in feature engineering
    - Is there any other features you can potentially create from existing features in the dataset
3. Remove features that are less important
    - Does removing features help reduce overfitting in the model?
4. Ensemble modelling
    - This is a more advanced technique whereby you combine prediction results from multiple machine learning models

# 9. Conclusion

You should achieve a submission score of 0.77511 if you follow exactly what I have done in this notebook. In other words, I have correctly predicted 77.5% of the test set. I highly encourage you to work through this project again and see if you can improve on this result.

If you found any mistakes in the notebook or places where I can potentially improve on, feel free to reach out to me. Let's help each other get better - happy learning!

My platforms: 
- [Facebook](https://www.facebook.com/chongjason914)
- [Instagram](https://www.instagram.com/chongjason914)
- [Twitter](https://www.twitter.com/chongjason914)
- [LinkedIn](https://www.linkedin.com/in/chongjason914)
- [YouTube](https://www.youtube.com/channel/UCQXiCnjatxiAKgWjoUlM-Xg?view_as=subscriber)
- [Medium](https://www.medium.com/@chongjason)

## References
https://github.com/chongjason914/kaggle-titanic 

https://www.kaggle.com/code/ldfreeman3/a-data-science-framework-to-achieve-99-accuracy

https://github.com/murilogustineli/Titanic-Classification

https://www.kaggle.com/code/kenjee/titanic-project-example/notebook

